In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image

In [ ]:
# preloading vgg19
model = models.vgg19(pretrained =True).features
# Assigning GPU 
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 110MB/s]


In [ ]:
#defing a function that will load the image and perform the required preprocessing and put it on the GPU
def image_loader(path):
    image=Image.open(path)
    #defining the image transformation steps to be performed before feeding them to the model
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    #The preprocessing steps involves resizing the image and then converting it to a tensor
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

#Loading the original and the style image
original_image=image_loader('Nikola-Tesla.jpg')
style_image=image_loader('style Image.jpg')

#Creating the generated image from the original image
generated_image=original_image.clone().requires_grad_(True)

In [ ]:
#Defining a class that for the model
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28'] 
        #Since we need only the 5 layers in the model so we will be dropping all the rest layers from the features of the model
        self.model=models.vgg19(pretrained=True).features[:29] #model will contain the first 29 layers
    
   
    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        #initialize an array that wil hold the activations from the chosen layers
        features=[]
        #Iterate over all the layers of the mode
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

In [ ]:
def calc_content_loss(gen_feat,orig_feat):

  content_l = torch.mean((gen_feat-orig_feat)**2)
  return content_l

In [ ]:
def calc_style_loss(gen,style):
  # calculate the gram matrix with style and generated image
  batch_size,channel,height,width = gen.shape

  G = torch.mm(gen.view(channel,height* width),gen.view(channel,height*width).t())
  A = torch.mm(style.view(channel,height* width),style.view(channel,height*width).t())
  # calculate style loss of each layer

  style_l = torch.mean((G-A)**2)
  return style_l

In [ ]:
def calculate_loss(gen_features, orig_features, style_features):
  style_loss = content_loss = 0

  for gen,cont, style in zip(gen_features,orig_features,style_features):
    content_loss += calc_content_loss(gen,cont)
    style_loss += calc_style_loss(gen,style)

  # calculating the total loss of the epoch
  total_loss = alpha*content_loss +beta*style_loss
  return total_loss

In [ ]:
#Load the model to the GPU
model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch=7000
lr=0.004
alpha=8
beta=70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=optim.Adam([generated_image],lr=lr)

In [ ]:
for e in range(epoch):
  gen_features = model(generated_image)
  orig_features = model(original_image)
  style_features = model(style_image)

  total_loss = calculate_loss(gen_features,orig_features,style_features)

  optimizer.zero_grad()
  total_loss.backward()

  optimizer.step()

  if(e/100):
    print(total_loss)

    save_image(generated_image,"gen.png")

tensor(5.9008e+09, grad_fn=<AddBackward0>)
tensor(5.0307e+09, grad_fn=<AddBackward0>)
tensor(4.2468e+09, grad_fn=<AddBackward0>)
tensor(3.6002e+09, grad_fn=<AddBackward0>)
tensor(3.0786e+09, grad_fn=<AddBackward0>)
tensor(2.6581e+09, grad_fn=<AddBackward0>)
tensor(2.3021e+09, grad_fn=<AddBackward0>)
tensor(2.0152e+09, grad_fn=<AddBackward0>)
